太棒了，高朋。让我们正式开启 RAG 的第一天。

这一步看似简单（只是切字符串），但它是整个 RAG 系统的**地基**。如果地基打得太碎（Chunk 太小），模型这就好比在玩“拼图猜画”，只见局部不见全貌；如果地基太大（Chunk 太大），又包含了太多噪音，检索精度会直线下降。

这是为您定制的 **S3W9D1** 学习内容。

---

## 🛠️ 今日工作区准备 (Environment Prep)

首先，我们需要更新工程结构，并安装必要的库。

**1. 目录创建**
在你的终端执行：

```bash
# 创建 Week 9 实验目录
mkdir -p stage3/week9

# 创建 RAG 源代码包（如果还没创建）
mkdir -p src/rag
touch src/rag/__init__.py

```

**2. 依赖安装**
我们需要 LangChain 的文本处理组件：

```bash
pip install langchain langchain-text-splitters

```

---

# S3W9D1: 文本切分策略 (Chunking Strategies)

**今日目标**：
理解并实现基于 `RecursiveCharacterTextSplitter` 的文本切分，掌握 `Chunk Size` 和 `Chunk Overlap` 对信息完整性的影响。

## 1. 理论知识讲解 (CS Principle)

在计算机科学中，Chunking 本质上是一个**粒度（Granularity）优化问题**。

### 核心概念：

1. **Context Window (上下文窗口)**：
* 虽然现在的 LLM (如 GPT-4-Turbo) 支持 128k 甚至更长，但**"Lost in the Middle" (中间迷失)** 现象依然存在。
* **RAG 的目的**：不是把整本书塞进去，而是通过检索，筛选出**Top-K** 最相关的片段，构建一个**高信噪比 (High Signal-to-Noise Ratio)** 的 Prompt。


2. **Trade-off (权衡)**：
* **Small Chunk (e.g., 128 tokens)**:
* *Pros*: 语义聚焦，向量表达非常精准。
* *Cons*: 容易切断逻辑，缺乏上下文。例如只切到“他被判刑了”，但不知道“他”是谁。


* **Large Chunk (e.g., 1024 tokens)**:
* *Pros*: 包含完整的上下文（主谓宾、因果关系）。
* *Cons*: 包含噪音，Embedding 向量会被无关信息稀释。




3. **Overlap (重叠/滑动窗口)**：
* 为了防止关键信息（如“张三因盗窃罪被判处...”）恰好被切分刀切在中间，我们需要引入 Overlap。
* 这类似于卷积神经网络（CNN）中的 **Stride < Kernel Size**，保证边界特征被多次采样。



---

## 2. 代码实现 (Code Implementation)

我们将对比“简单字符切分”和“递归字符切分”。对于法律文档，**保持条款的完整性**至关重要。

请在 `stage3/week9/S3W9D1_Chunking_Exp.ipynb` 中运行以下代码：

### 2.1 准备测试数据 (模拟法律文本)

```python
# 模拟一段有层级结构的法律文本
# 注意：这里有明确的段落结构（\n\n）和句子结构（。）
legal_text = """
第一条 为了惩罚犯罪，保护人民，根据宪法，结合我国同犯罪作斗争的具体经验及实际情况，制定本法。

第二条 中华人民共和国刑法任务。
中华人民共和国刑法的任务，是用刑罚同一切犯罪行为作斗争，以保卫国家安全，保卫人民民主专政的政权和社会主义制度，保护国有财产和劳动群众集体所有的财产，保护公民私人所有的财产，保护公民的人身权利、民主权利和其他权利，维护社会秩序、经济秩序，保障社会主义建设事业的顺利进行。

第三条 罪刑法定。
法律明文规定为犯罪行为的，依照法律定罪处刑；法律没有明文规定为犯罪行为的，不得定罪处刑。
"""

```

### 2.2 核心切分逻辑 (Recursive Splitter)

这是目前业界最常用的切分器。

```python
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 定义一个辅助函数来展示切分结果
def print_chunks(chunks):
    print(f"--- Total Chunks: {len(chunks)} ---")
    for i, chunk in enumerate(chunks):
        print(f"[Chunk {i} | Length: {len(chunk)}]:")
        print(chunk)
        print("-" * 20)

# 实验 1: 激进切分 (Chunk太小，容易丢失语义)
print("=== Experiment 1: Small Chunk Size (100 chars) ===")
splitter_small = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    separators=["\n\n", "\n", "。", ""] # 优先级：先按段落切，再按换行，再按句号
)
chunks_small = splitter_small.split_text(legal_text)
print_chunks(chunks_small)

# 实验 2: 推荐配置 (适合法律条文，保留一定上下文)
print("\n=== Experiment 2: Optimal Chunk Size (200 chars + Overlap) ===")
splitter_optimal = RecursiveCharacterTextSplitter(
    chunk_size=200,      # 大致相当于一个完整的法律条款长度
    chunk_overlap=50,    # 50字符的重叠，防止“第三条”标题和内容分家
    separators=["\n\n", "\n", "。", ""] 
)
chunks_optimal = splitter_optimal.split_text(legal_text)
print_chunks(chunks_optimal)

```

**观察重点**：

* 在实验 1 中，你可能会发现某个法律条文被强行腰斩。
* 在实验 2 中，注意观察 `chunk_overlap` 带来的重复文本，这部分重复就是为了保证语义连续性。

---

## 3. 深度理论 (Deep Dive)

为什么我们首选 `RecursiveCharacterTextSplitter`？

它的算法逻辑是一种 **Greedy Reduction (贪婪归约)** 策略。
它接收一个 `separators` 列表，例如 `["\n\n", "\n", " ", ""]`。

1. **Level 1**: 尝试用第一个分隔符（`\n\n`，即段落）切分。
2. **Check**: 检查切分出来的每一块，长度是否超过 `chunk_size`？
* 如果**没有超过**：保留该块。
* 如果**超过了**：对这个特定的块，进入 **Level 2**，用第二个分隔符（`\n`，即行）继续切分。


3. **Recursion**: 重复上述过程，直到所有块都满足长度要求。

**CS 启示**：
这体现了**分治算法 (Divide and Conquer)** 的思想，但增加了一个约束条件（尽可能保持高层级的语义完整性）。对于代码文件，这个分隔符列表会变成 `["class ", "def ", "\n"]`，原理是一样的。

---

## 4. 工程封装 (Engineering Encapsulation)

在 Jupyter Notebook 跑通后，我们需要将其封装为工程代码，供后续 Pipeline 调用。

**创建/编辑文件**：`src/rag/etl.py`

```python
from typing import List
from langchain_text_splitters import RecursiveCharacterTextSplitter

class TextChunker:
    """
    RAG ETL Pipeline 中的切分模块
    """
    def __init__(self, chunk_size: int = 500, chunk_overlap: int = 50):
        self.splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            # 针对中文优化分隔符
            separators=["\n\n", "\n", "。", "！", "？", "，", " ", ""]
        )

    def split(self, text: str) -> List[str]:
        """
        输入原始长文本，输出文本块列表
        """
        if not text:
            return []
        return self.splitter.split_text(text)

# 使用示例 (便于测试)
if __name__ == "__main__":
    text = "测试文本..." * 50
    chunker = TextChunker(chunk_size=100, chunk_overlap=20)
    chunks = chunker.split(text)
    print(f"Generated {len(chunks)} chunks.")

```

---

## 5. 模拟面试 (Mock Interview)

**面试官**：*“在处理法律文档时，Chunk Size 应该设大一点还是小一点？为什么？”*

**参考回答范式**：

> “这取决于两个因素：**Embedding 模型的输入限制**和**数据的内在逻辑结构**。
> 1. **语义完整性**：法律文档通常以‘条’或‘款’为单位。如果 Chunk Size 太小（如 100），可能会把‘犯罪行为’和‘量刑标准’切开，导致检索时丢失关联。
> 2. **噪声控制**：如果 Chunk Size 太大（如 2000），一个 Chunk 可能包含多个不相关的法条。当用户问‘盗窃罪’时，检索出的 Chunk 里如果混杂了‘抢劫罪’的描述，会干扰 LLM 的生成，产生幻觉。
> 
> 
> **结论**：对于法律场景，我通常会选择 **512 左右的 Token 数**（约 800-1000 中文字符），并配合 **10%-20% 的 Overlap**。同时，使用递归切分器优先保证在段落（`\n\n`）处切断，确保法条的原子性。”

---

## 6. 今日 LeetCode 练习

* **题目**：[206. Reverse Linked List (反转链表)](https://leetcode.com/problems/reverse-linked-list/)
* **关联**：链表操作是很多高级数据结构（如图的邻接表、LRU缓存）的基础。
* **要求**：请分别尝试 **迭代 (Iterative)** 和 **递归 (Recursive)** 两种解法。

```python
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

class Solution:
    def reverseList(self, head: ListNode) -> ListNode:
        # 迭代法 (推荐面试用，空间复杂度 O(1))
        prev = None
        curr = head
        while curr:
            next_temp = curr.next  # 暂存下一步
            curr.next = prev       # 反转指针
            prev = curr            # 步进 prev
            curr = next_temp       # 步进 curr
        return prev

```

---

## 7. 今日任务总结

* [ ] 建立了 `src/rag` 目录结构。
* [ ] 理解了 `RecursiveCharacterTextSplitter` 的“贪婪归约”逻辑。
* [ ] 封装了 `TextChunker` 类。
* [ ] 完成了 LeetCode 206。

**准备好进入 Day 2 了吗？** 明天我们将把这些切分好的文本块，扔进高维空间，看看它们变成向量后长什么样。